# Initial Settings

In [40]:
import pandas as pd
import numpy as np
import geopandas as gpd
import jalali
import statistics
import plotly.graph_objects as go
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import voronoi_diagram as svd
from shapely.wkt import loads as load_wkt
from datetime import datetime



# Modify Date And Water Table
# Date Type: 
#   1.Gregorian (Text): 2000-11-1
#   2.Persian (Text): 1399-1-1
# Date & Value:
#   Column of Pandas Dataframe
def convert_to_day_15(info, date, value, date_type="persian"):
    """[summary]

    Args:
        info ([type]): [description]
        date ([type]): [description]
        value ([type]): [description]
        date_type (str, optional): [description]. Defaults to "persian".

    Returns:
        [type]: [description]
    """
    df = info.copy()
    if date_type == "gregorian":
        df["DATE_GREGORIAN"] = date.apply(pd.to_datetime)
        df["DATE_PERSIAN"] = list(
            map(
                lambda i: jalali.Gregorian(i.date()).persian_string(),
                df["DATE_GREGORIAN"]
            )
        )
    elif date_type == "persian":
        df["DATE_PERSIAN"] = date
        df["DATE_GREGORIAN"] = list(
            map(
                lambda i: jalali.Persian(i).gregorian_string(),
                df["DATE_PERSIAN"]
            )
        )
        df["DATE_GREGORIAN"] = df["DATE_GREGORIAN"].apply(pd.to_datetime)    
    else:
        pass
    df["VALUE"] = value        
    df["DELTA_DAY"] = df["DATE_GREGORIAN"].diff().dt.days
    df["DATE_PERSIAN_NEW"] = list(
        map(
            lambda i: f"{int(i.split('-')[0])}-{int(i.split('-')[1])}-{15}",
            df["DATE_PERSIAN"]
        )
    )
    df["DATE_GREGORIAN_NEW"] = list(
        map(
            lambda i: jalali.Persian(i).gregorian_string(),
            df["DATE_PERSIAN_NEW"]
        )
    )
    df["DATE_GREGORIAN_NEW"] = df["DATE_GREGORIAN_NEW"].apply(pd.to_datetime)
    df["VALUE_NEW"] = df["VALUE"]
    A = []
    A.append(df["VALUE"][0])
    for i in range(1, len(df)):
        if int(df["DATE_PERSIAN"][i].split('-')[2]) >= 15:
            NEW_VALUE = df["VALUE"][i-1] + ((((df["DATE_GREGORIAN_NEW"][i] - df["DATE_GREGORIAN"][i-1]).days) / ((df["DATE_GREGORIAN"][i] - df["DATE_GREGORIAN"][i-1]).days)) * (df["VALUE"][i] - df["VALUE"][i-1]))
            A.append(NEW_VALUE)
        else:
            NEW_VALUE = df["VALUE"][i] + ((((df["DATE_GREGORIAN_NEW"][i] - df["DATE_GREGORIAN"][i]).days) / ((df["DATE_GREGORIAN"][i+1] - df["DATE_GREGORIAN"][i]).days)) * (df["VALUE"][i+1] - df["VALUE"][i]))
            A.append(NEW_VALUE)
    df["VALUE_NEW"] = A
    return df


def create_date_day15(min, max):
    """[summary]

    Args:
        min ([type]): [description]
        max ([type]): [description]

    Returns:
        [type]: [description]
    """
    result = []
    min_list = list(map(lambda x: int(x), min.split("-")))
    max_list = list(map(lambda x: int(x), max.split("-")))
    for y in range(min_list[0], max_list[0] + 1):
        for m in range(1, 13):
            result.append(f"{y}-{m}-15")

    result = pd.DataFrame(
        {"DATE_PERSIAN" : result}
    )
    result['DATE_GREGORIAN'] = result.apply(
        lambda x: jalali.Persian(x["DATE_PERSIAN"]).gregorian_string(), 
        axis=1
    )
    result["DATE_GREGORIAN"] = result["DATE_GREGORIAN"].apply(pd.to_datetime)
    result = result[result["DATE_GREGORIAN"] >= pd.to_datetime(jalali.Persian(min).gregorian_string())]
    result = result[result["DATE_GREGORIAN"] <= pd.to_datetime(jalali.Persian(max).gregorian_string())]
    result["DATE_GREGORIAN"] = result["DATE_GREGORIAN"].apply(pd.to_datetime)  
    return result


def dropHolesBase(plg):

	'''

	BASIC FUNCTION TO REMOVE / DROP / FILL THE HOLES.

	PARAMETERS:

		plg: plg WHO HAS HOLES / EMPTIES.
			Type: shapely.geometry.MultiPolygon OR shapely.geometry.Polygon

	RETURNS:
		A shapely.geometry.MultiPolygon OR shapely.geometry.Polygon object
	

	'''

	if isinstance(plg, MultiPolygon):

		return MultiPolygon(Polygon(p.exterior) for p in plg)

	elif isinstance(plg, Polygon):

		return Polygon(plg.exterior)


def dropHoles(gdf):

	'''

	REMOVE / DROP / FILL THE HOLES / EMPTIES FOR ITERMS IN GeoDataFrame.
	
	PARAMETERS:
		gdf:
			Type: geopandas.GeoDataFrame

	RETURNS:
		gdf_nohole: GeoDataFrame WITHOUT HOLES
			Type: geopandas.GeoDataFrame
	
	'''

	gdf_nohole = gpd.GeoDataFrame()

	for g in gdf['geometry']:

		geo = gpd.GeoDataFrame(geometry=gpd.GeoSeries(dropHolesBase(g)))

		gdf_nohole=gdf_nohole.append(geo,ignore_index=True)

	gdf_nohole.rename(columns={gdf_nohole.columns[0]:'geometry'}, inplace=True)

	gdf_nohole.crs = gdf.crs

	gdf.rename(columns={'geometry': 'geometry_old'}, inplace=True)

	gdf["geometry_new"] = gdf_nohole

	gdf.rename(columns={'geometry_new': 'geometry'}, inplace=True)

	gdf.drop(['geometry_old'], axis=1, inplace=True)

	return gdf


def thiessen_polygons(gdf, mask):

	'''

	CREATE VORONOI DIAGRAM / THIESSEN POLYGONS:

	PARAMETERS:

		gdf: POINTS / POLYGONS TO BE USED TO CREATE VORONOI DIAGRAM / THIESSEN POLYGONS.
            Type: geopandas.GeoDataFrame

		mask: POLYGON VECTOR USED TO CLIP THE CREATED VORONOI DIAGRAM / THIESSEN POLYGONS.
			Type: GeoDataFrame, GeoSeries, (Multi)Polygon

	RETURNS:

		gdf_vd: THIESSEN POLYGONS
			Type: geopandas.geodataframe.GeoDataFrame
	
	'''
	
	gdf.reset_index(drop=True)

	# CONVERT TO shapely.geometry.MultiPolygon
	smp = gdf.unary_union

	# CREATE PRIMARY VORONOI DIAGRAM BY INVOKING shapely.ops.voronoi_diagram
	poly = load_wkt('POLYGON ((42 24, 64 24, 64 42, 42 42, 42 24))')
	smp_vd = svd(smp, envelope=poly)

	# CONVERT TO GeoSeries AND explode TO SINGLE POLYGONS
	gs = gpd.GeoSeries([smp_vd]).explode()

	# CONVERT TO GEODATAFRAME
	# NOTE THAT IF GDF WAS shapely.geometry.MultiPolygon, IT HAS NO ATTRIBUTE 'crs'
	gdf_vd_primary = gpd.geodataframe.GeoDataFrame(geometry=gs, crs=gdf.crs)
	
	# RESET INDEX
	gdf_vd_primary.reset_index(drop=True)
	
	# SPATIAL JOIN BY INTERSECTING AND DISSOLVE BY `index_right`
	gdf_temp = (gpd.sjoin(gdf_vd_primary, gdf, how='inner', op='intersects').dissolve(by='index_right').reset_index(drop=True))

	gdf_vd = gpd.clip(gdf_temp, mask)

	gdf_vd = dropHoles(gdf_vd)

	return gdf_vd

# Read Data

In [41]:
# Read Data And GeoInfo
xls = pd.ExcelFile('Data/CSV/HydrographData.xlsx')
Data = pd.read_excel(xls, sheet_name='Data')
GeoInfo = pd.read_excel(xls, sheet_name='GeoInfo')

# GeoInfo
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.rstrip())
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.lstrip())

# Data
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
Data[COLs] = Data[COLs].apply(lambda x: x.str.rstrip())
Data[COLs] = Data[COLs].apply(lambda x: x.str.lstrip())

gdf = gpd.read_file("Data/GeoJson/Sarakhs_Points.geojson")
gdf = gdf.set_crs("EPSG:32641", allow_override=True)

mask = gpd.read_file("Data/GeoJson/Sarakhs_Aquifer.geojson")
mask = mask.set_crs("EPSG:32641", allow_override=True)

# Convert Date

In [42]:
Data["DATE_GREGORIAN_RAW"] = Data["DATE_GREGORIAN_RAW"].apply(pd.to_datetime)

Data['DATE_CHECK'] = np.where(
    Data["DATE_PERSIAN_RAW"].isna(),
    np.where(
        Data["DATE_GREGORIAN_RAW"].isna(),
        np.NaN,
        "G"
    ),
    "P"  
)

Data['DATE_PERSIAN_RAW'] = Data.apply(
    lambda x: jalali.Gregorian(x["DATE_GREGORIAN_RAW"].date()).persian_string() if x["DATE_CHECK"] == "G" else x["DATE_PERSIAN_RAW"], 
    axis=1
)

Data['DATE_GREGORIAN_RAW'] = Data.apply(
    lambda x: jalali.Persian(x["DATE_PERSIAN_RAW"]).gregorian_string() if x["DATE_CHECK"] == "P" else x["DATE_GREGORIAN_RAW"], 
    axis=1
)

Data["DATE_GREGORIAN_RAW"] = Data["DATE_GREGORIAN_RAW"].apply(pd.to_datetime)


Data.drop(['DATE_CHECK'], axis=1, inplace=True)

Data.sort_values(
    by=["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_GREGORIAN_RAW"], 
    inplace=True
)

# Gap Filling

## Convert To Day 15

In [43]:
# Remove NanN Data In Column "WATER_TABLE_RAW"
Data.dropna(
    subset=["WATER_TABLE_RAW"],
    inplace=True
)

Data.reset_index(
    inplace=True,
    drop=True
)

wt_date_converted = convert_to_day_15(
    info=Data[["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME"]],
    date=Data["DATE_PERSIAN_RAW"],
    value=Data["WATER_TABLE_RAW"],
    date_type="persian"
)[["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN", "DATE_PERSIAN_NEW", "DATE_GREGORIAN", "DATE_GREGORIAN_NEW", "VALUE_NEW"]]

wt_date_converted.columns = ["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN_RAW", "DATE_PERSIAN", "DATE_GREGORIAN_RAW","DATE_GREGORIAN", "WATER_TABLE"]

Data = Data.merge(
    right=wt_date_converted,
    how="left",
    on=["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN_RAW", "DATE_GREGORIAN_RAW"]
)

## Interpolate

In [44]:
tmp = pd.DataFrame()

for mn in list(Data["MAHDOUDE_NAME"].unique()):
    for an in list(Data["AQUIFER_NAME"].unique()):
        for ln in list(Data["LOCATION_NAME"].unique()):
            df = Data[(Data["MAHDOUDE_NAME"] == mn) & (Data["AQUIFER_NAME"] == an) & (Data["LOCATION_NAME"] == ln)]
            date = create_date_day15(
                min = df.DATE_PERSIAN.min(),
                max = df.DATE_PERSIAN.max()
            )
            
            df = date.merge(
                df,
                how="left",
                on=["DATE_PERSIAN", "DATE_GREGORIAN"]
            )
            df["WATER_TABLE_INTERPOLATE"] = df["WATER_TABLE"].interpolate(method='akima')
            # df["WATER_TABLE_LINEAR"] = df["WATER_TABLE_LINEAR"].interpolate(method='ffill', limit=3)
            # df["WATER_TABLE_LINEAR"] = df["WATER_TABLE_LINEAR"].interpolate(method='bfill', limit=3)
            # df["WATER_TABLE_SPLINE"] = df["WATER_TABLE"].interpolate(method='akima')
            # df["WATER_TABLE_SPLINE"] = df["WATER_TABLE_SPLINE"].interpolate(method='ffill', limit=3)
            # df["WATER_TABLE_SPLINE"] = df["WATER_TABLE_SPLINE"].interpolate(method='bfill', limit=3)
            df["MAHDOUDE_NAME"] = mn
            df["MAHDOUDE_CODE"] = int(df["MAHDOUDE_CODE"].unique()[0])
            df["AQUIFER_NAME"] = an
            df["LOCATION_NAME"] = ln
            df["DATA_STATE"] = df["DATA_STATE"].fillna("M")
            df["STORAGE_COEFFICIENT_LOCATION"] = df["STORAGE_COEFFICIENT_LOCATION"].unique()[0]
            
            df = df[[
                "MAHDOUDE_NAME", "MAHDOUDE_CODE", "AQUIFER_NAME", "LOCATION_NAME",
                "DATE_GREGORIAN", "DATE_PERSIAN",
                "WATER_TABLE", "WATER_TABLE_INTERPOLATE", "STORAGE_COEFFICIENT_LOCATION", "THISSEN_LOCATION", "THISSEN_AQUIFER",
                "DATA_STATE", "NO_MEASURE_CODE", "INFO",
                "DATE_GREGORIAN_RAW", "DATE_PERSIAN_RAW", "WATER_TABLE_RAW"	
            ]]

            tmp = pd.concat([tmp, df], axis=0)

Data = tmp.copy()

del tmp

# Thissen Calculation

In [45]:
# SELECT LEVEL
SELECTED_LEVEL = 'LEVEL_SRTM'

# FILTER DATE
Data = Data[Data["DATE_GREGORIAN"] >= datetime.strptime("2001-10-01", '%Y-%m-%d')]

def f (df_raw):
    df = df_raw.dropna(subset=["WATER_TABLE"])    
    vd_result = gpd.GeoDataFrame()    
    gdf_d = gdf[gdf["LOCATION_NAME"].isin(df["LOCATION_NAME"])]    
    # TODO: Find Better Way
    for i in range(len(mask)):
        mask_tmp = mask.iloc[[i]]
        gdf_d['CHECK'] = gdf_d['geometry'].apply(lambda x: mask_tmp.contains(x))
        gdf_tmp = gdf_d[gdf_d['CHECK']].reset_index(drop=True)        
        if len(gdf_tmp) > 0:
            vd = thiessen_polygons(gdf_tmp, mask_tmp)
            vd.set_geometry(col='geometry', inplace=True)
            vd.set_crs("EPSG:32641", allow_override=True, inplace=True)
            vd["THISSEN_LOCATION"] = vd.geometry.area / 1000000
            vd["THISSEN_AQUIFER"] = [mask_tmp.geometry.area[0] / 1000000] * len(gdf_tmp)
            vd["DATE_PERSIAN"] = [df["DATE_PERSIAN"].unique()[0]] * len(gdf_tmp)
            vd_result = vd_result.append(vd, ignore_index=True)                
    return vd_result

def f_interpolate (df_raw):
    df = df_raw.dropna(subset=["WATER_TABLE_INTERPOLATE"])
    vd_result = gpd.GeoDataFrame()    
    gdf_d = gdf[gdf["LOCATION_NAME"].isin(df["LOCATION_NAME"])]    
    # TODO: Find Better Way
    for i in range(len(mask)):
        mask_tmp = mask.iloc[[i]]
        gdf_d['CHECK'] = gdf_d['geometry'].apply(lambda x: mask_tmp.contains(x))
        gdf_tmp = gdf_d[gdf_d['CHECK']].reset_index(drop=True)        
        if len(gdf_tmp) > 0:
            vd = thiessen_polygons(gdf_tmp, mask_tmp)
            vd.set_geometry(col='geometry', inplace=True)
            vd.set_crs("EPSG:32641", allow_override=True, inplace=True)
            vd["THISSEN_LOCATION"] = vd.geometry.area / 1000000
            vd["THISSEN_AQUIFER"] = [mask_tmp.geometry.area[0] / 1000000] * len(gdf_tmp)
            vd["DATE_PERSIAN"] = [df["DATE_PERSIAN"].unique()[0]] * len(gdf_tmp)
            vd_result = vd_result.append(vd, ignore_index=True)                
    return vd_result

vd_result = Data.groupby(
    by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'],
).apply(f)

vd_result = vd_result.droplevel(['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])


vd_result_interpolate = Data.groupby(
    by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'],
).apply(f_interpolate)

vd_result_interpolate = vd_result_interpolate.droplevel(['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])


Data_Original = Data.merge(
    right=vd_result[['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME', 'DATE_PERSIAN', SELECTED_LEVEL, 'THISSEN_LOCATION', 'THISSEN_AQUIFER', 'geometry']],
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME', 'DATE_PERSIAN']
).sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN']).rename(columns={SELECTED_LEVEL: 'LEVEL'})

# THISSEN
Data_Original["THISSEN_LOCATION"] = np.where(Data_Original["THISSEN_LOCATION_x"].isna(), Data_Original["THISSEN_LOCATION_y"], Data_Original["THISSEN_LOCATION_x"])
Data_Original["THISSEN_AQUIFER"] = np.where(Data_Original["THISSEN_AQUIFER_x"].isna(), Data_Original["THISSEN_AQUIFER_y"], Data_Original["THISSEN_AQUIFER_x"])

Data_Interpolate = Data.merge(
    right=vd_result_interpolate[['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME', 'DATE_PERSIAN', SELECTED_LEVEL, 'THISSEN_LOCATION', 'THISSEN_AQUIFER', 'geometry']],
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME', 'DATE_PERSIAN']
).sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN']).rename(columns={SELECTED_LEVEL: 'LEVEL'})

# THISSEN
Data_Interpolate["THISSEN_LOCATION"] = np.where(Data_Interpolate["THISSEN_LOCATION_x"].isna(), Data_Interpolate["THISSEN_LOCATION_y"], Data_Interpolate["THISSEN_LOCATION_x"])
Data_Interpolate["THISSEN_AQUIFER"] = np.where(Data_Interpolate["THISSEN_AQUIFER_x"].isna(), Data_Interpolate["THISSEN_AQUIFER_y"], Data_Interpolate["THISSEN_AQUIFER_x"])

# Calculate Head Aquifer: Original

In [46]:
# HEAD LOCATION
Data_Original['HEAD_LOCATION'] = Data_Original['LEVEL'] - Data_Original['WATER_TABLE']


# HEAD AQUIFER ARITHMETIC, GEOMETRIC, HARMONIC
tmp = Data_Original[['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_LOCATION']].dropna(subset=['HEAD_LOCATION']).reset_index()\
    .groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])\
        .agg({'HEAD_LOCATION': [statistics.mean]})\
            .reset_index()

tmp.columns = [col for col in tmp.columns]
tmp.columns = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_AQUIFER_ARITHMETIC']
# tmp.columns = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_AQUIFER_ARITHMETIC', 'HEAD_AQUIFER_GEOMETRIC', 'HEAD_AQUIFER_HARMONIC']

Data_Original = Data_Original.merge(
    right=tmp,
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])

del tmp


# HEAD AQUIFER THISSEN
Data_Original['TMP'] = (Data_Original['HEAD_LOCATION'] * Data_Original['THISSEN_LOCATION']) / Data_Original['THISSEN_AQUIFER']

tmp = Data_Original.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).sum().reset_index()[
            ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN', 'TMP']].rename(columns={'TMP': 'HEAD_AQUIFER_THISSEN'})

# TODO: Find Better Way: Now, Replace All Zero with NaN
tmp['HEAD_AQUIFER_THISSEN'] = tmp['HEAD_AQUIFER_THISSEN'].replace({'0': np.nan, 0: np.nan})

Data_Original = Data_Original.merge(
    right=tmp,
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN'])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])\
            .drop(['TMP'], axis=1)

del tmp

# Calculate Head Aquifer: Interpolate

In [47]:
# HEAD LOCATION
Data_Interpolate['HEAD_LOCATION'] = Data_Interpolate['LEVEL'] - Data_Interpolate['WATER_TABLE_INTERPOLATE']


# HEAD AQUIFER ARITHMETIC, GEOMETRIC, HARMONIC
tmp = Data_Interpolate[['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_LOCATION']].dropna(subset=['HEAD_LOCATION']).reset_index()\
    .groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])\
        .agg({'HEAD_LOCATION': [statistics.mean]})\
            .reset_index()

tmp.columns = [col for col in tmp.columns]
tmp.columns = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_AQUIFER_ARITHMETIC']
# tmp.columns = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_AQUIFER_ARITHMETIC', 'HEAD_AQUIFER_GEOMETRIC', 'HEAD_AQUIFER_HARMONIC']

Data_Interpolate = Data_Interpolate.merge(
    right=tmp,
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])

del tmp


# HEAD AQUIFER THISSEN
Data_Interpolate['TMP'] = (Data_Interpolate['HEAD_LOCATION'] * Data_Interpolate['THISSEN_LOCATION']) / Data_Interpolate['THISSEN_AQUIFER']

tmp = Data_Interpolate.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).sum().reset_index()[
            ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN', 'TMP']].rename(columns={'TMP': 'HEAD_AQUIFER_THISSEN'})

# TODO: Find Better Way: Now, Replace All Zero with NaN
tmp['HEAD_AQUIFER_THISSEN'] = tmp['HEAD_AQUIFER_THISSEN'].replace({'0': np.nan, 0: np.nan})

Data_Interpolate = Data_Interpolate.merge(
    right=tmp,
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN'])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])\
            .drop(['TMP'], axis=1)

del tmp

# Storage Coefficient: Original

In [48]:
# STORAGE COEFFICIENT AQUIFER
Data_Original['TMP'] = (Data_Original['STORAGE_COEFFICIENT_LOCATION'] * Data_Original['THISSEN_LOCATION']) / Data_Original['THISSEN_AQUIFER']

tmp = Data_Original.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).sum().reset_index()[
            ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN', 'TMP']].rename(columns={'TMP': 'STORAGE_COEFFICIENT_AQUIFER'})

Data_Original = Data_Original.merge(
    right=tmp,
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN'])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])\
            .drop(['TMP'], axis=1)

del tmp

# Storage Coefficient: Interpolate

In [49]:
# STORAGE COEFFICIENT AQUIFER
Data_Interpolate['TMP'] = (Data_Interpolate['STORAGE_COEFFICIENT_LOCATION'] * Data_Interpolate['THISSEN_LOCATION']) / Data_Interpolate['THISSEN_AQUIFER']

tmp = Data_Interpolate.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).sum().reset_index()[
            ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN', 'TMP']].rename(columns={'TMP': 'STORAGE_COEFFICIENT_AQUIFER'})

Data_Interpolate = Data_Interpolate.merge(
    right=tmp,
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN'])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])\
            .drop(['TMP'], axis=1)

del tmp

# Adjust

In [139]:
data = Data_Original.copy()
data = data.dropna(subset=["WATER_TABLE"])

data = data[[
    'MAHDOUDE_NAME', 'MAHDOUDE_CODE', 'AQUIFER_NAME', 'LOCATION_NAME',
    'DATE_GREGORIAN', 'DATE_PERSIAN', 'WATER_TABLE',
    'WATER_TABLE_INTERPOLATE', 'STORAGE_COEFFICIENT_LOCATION',
    'LEVEL', 'THISSEN_LOCATION', 'THISSEN_AQUIFER', 'HEAD_LOCATION',
    'HEAD_AQUIFER_ARITHMETIC', 'HEAD_AQUIFER_THISSEN',
    'STORAGE_COEFFICIENT_AQUIFER'
]]

CHECK_LOCATION = data.groupby(by=["DATE_GREGORIAN", "DATE_PERSIAN"])["LOCATION_NAME"].apply(list).reset_index(name='LOCATION_LIST').sort_values(['DATE_GREGORIAN'])

CHECK_LOCATION_CONDI = []

for i, l in enumerate(CHECK_LOCATION.LOCATION_LIST):
    if i == 0:
        CHECK_LOCATION_CONDI.append(False)
    elif set(CHECK_LOCATION.LOCATION_LIST[i]) == set(CHECK_LOCATION.LOCATION_LIST[i-1]):
        CHECK_LOCATION_CONDI.append(False)
    else:
        CHECK_LOCATION_CONDI.append(True)


CHECK_LOCATION["CHECK_LOCATION_CONDI"] = CHECK_LOCATION_CONDI

data_aquifer = data.drop_duplicates(subset=['DATE_GREGORIAN', 'DATE_PERSIAN'], keep='last').reset_index()

data_aquifer = data_aquifer.merge(
    right=CHECK_LOCATION,
    how='left',
    on=["DATE_GREGORIAN", "DATE_PERSIAN"])\
        .sort_values(['DATE_GREGORIAN'])

data_aquifer = data_aquifer[[
    "DATE_GREGORIAN", "DATE_PERSIAN",
    "HEAD_AQUIFER_THISSEN", "CHECK_LOCATION_CONDI",
]]

data_aquifer

# data_aquifer.replace(0, np.nan, inplace=True)
# data_aquifer['Delta'] = data_aquifer['Aquifer_Head'].diff().fillna(0)
# data_aquifer['Index'] = abs(data_aquifer['Delta']).apply(lambda x: 1 if x >= threshold else 0)
# data_aquifer['Adjusted_Aquifer_Head'] = data_aquifer['Aquifer_Head']


# n = data_aquifer.index[data_aquifer['Index'] == True].tolist()


# if len(n) > 0:
#     while len(n) != 0:
#         delta = data_aquifer['Delta'][n[0]]
#         data_aquifer['Temp_Aquifer_Head'] = data_aquifer['Adjusted_Aquifer_Head']
#         for i in range(n[0]):
#             data_aquifer['Temp_Aquifer_Head'][i] = data_aquifer['Adjusted_Aquifer_Head'][i] + delta
#             data_aquifer['Adjusted_Aquifer_Head'] = data_aquifer['Temp_Aquifer_Head']
#             data_aquifer['Delta'] = data_aquifer['Adjusted_Aquifer_Head'].diff().fillna(0)
#             data_aquifer['Index'] = abs(data_aquifer['Delta']).apply(lambda x: 1 if x >= threshold else 0)
#             n = data_aquifer.index[data_aquifer['Index'] == True].tolist()
            

# if 'Temp_Aquifer_Head' in data_aquifer.columns:
#     data_aquifer = data_aquifer.drop(['Temp_Aquifer_Head'], axis=1)

# if 'Delta' in data_aquifer.columns:
#     data_aquifer = data_aquifer.drop(['Delta'], axis=1)

# if 'Index' in data_aquifer.columns:
#     data_aquifer = data_aquifer.drop(['Index'], axis=1)



,DATE_GREGORIAN,DATE_PERSIAN,HEAD_AQUIFER_THISSEN,CHECK_LOCATION_CONDI
5,2001-10-07,1380-7-15,288.253354,False
6,2001-11-06,1380-8-15,288.287947,False
7,2001-12-06,1380-9-15,288.316256,False
8,2002-01-05,1380-10-15,288.320160,False
9,2002-02-04,1380-11-15,288.365595,False
10,2002-03-06,1380-12-15,288.315207,False
11,2002-04-04,1381-1-15,288.241408,False
12,2002-05-05,1381-2-15,288.171136,False
13,2002-06-05,1381-3-15,288.100363,False
14,2002-07-06,1381-4-15,288.000966,False


In [140]:
df_original = Data_Original.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).mean().reset_index()
df_original = df_original[df_original["HEAD_AQUIFER_THISSEN"] >= 250]
df_interpolate = Data_Interpolate.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).mean().reset_index()

clc = data_aquifer[data_aquifer["CHECK_LOCATION_CONDI"]]
clc = clc[clc["HEAD_AQUIFER_THISSEN"] >= 250]
# Create Traces
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_interpolate["DATE_PERSIAN"],
        y=df_interpolate["HEAD_AQUIFER_THISSEN"],
        mode='lines+markers',
        name='با درون یابی',
        marker=dict(
            color='red',
            size=12,
        )
    )
)



fig.add_trace(
    go.Scatter(
        x=df_original["DATE_PERSIAN"],
        y=df_original["HEAD_AQUIFER_THISSEN"],
        mode='lines+markers',
        name='بدون درون یابی',
        marker=dict(
            color='black',
            size=12,
        )
    )
)



fig.add_trace(
    go.Scatter(
        x=clc["DATE_PERSIAN"],
        y=clc["HEAD_AQUIFER_THISSEN"],
        mode='markers',
        name='تغییر شبکه تیسن',
        marker=dict(
            color='green',
            size=12,
        )
    )
)






fig.update_layout(
    yaxis_title="HEAD AQUIFER (m)",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    ),
    xaxis=dict(
        tickformat="%Y-%m-%d"
    ),
)

fig.show()

In [118]:
a1 = Data_Original[Data_Original.DATE_PERSIAN == "1392-5-15"][["LOCATION_NAME", "WATER_TABLE", "HEAD_LOCATION", "THISSEN_LOCATION"]]
a2 = Data_Original[Data_Original.DATE_PERSIAN == "1392-6-15"][["LOCATION_NAME", "WATER_TABLE", "HEAD_LOCATION", "THISSEN_LOCATION"]]

a1.merge(
    right=a2,
    how="outer",
    on="LOCATION_NAME"
)

,LOCATION_NAME,WATER_TABLE_x,HEAD_LOCATION_x,THISSEN_LOCATION_x,WATER_TABLE_y,HEAD_LOCATION_y,THISSEN_LOCATION_y
0,اراضی حسن آباد جر -wa,NaN,NaN,NaN,NaN,NaN,NaN
1,اراضی حسن آباد جر -wb,24.357586,247.642414,23.630649,24.335789,247.664211,23.630649
2,اول کچولی,24.150645,265.849355,12.354495,24.306949,265.693051,12.354495
3,ایستگاه کنترل گاز,44.752857,302.247143,49.784891,44.879298,302.120702,49.784891
4,بابا لقمان,20.730000,248.270000,8.877529,20.974407,248.025593,8.877529
5,بعد از آصف- چاه شماره 5,20.554074,272.445926,12.723486,20.791724,272.208276,12.723486
6,بعد از قوش سربزی عیسی مختاری,18.833704,284.166296,23.505986,19.109310,283.890690,23.505986
7,بلوار امام رضا (ع),20.361212,253.638788,6.383106,20.491154,253.508846,6.383106
8,بین تام مختار و کندکلی,26.177586,241.822414,11.244847,26.600877,241.399123,11.244847
9,بین کچولی و قوش عظیم,18.593793,272.406207,11.931257,18.957193,272.042807,11.931257


In [54]:
for mn in list(Data_Original["MAHDOUDE_NAME"].unique()):
    for an in list(Data_Original["AQUIFER_NAME"].unique()):
        for ln in list(Data_Original["LOCATION_NAME"].unique()):

            df = Data_Original[(Data_Original["MAHDOUDE_NAME"] == mn) & (Data_Original["MAHDOUDE_NAME"] == an) & (Data_Original["LOCATION_NAME"] == ln)]

            # Create Traces
            fig = go.Figure()

            fig.add_trace(
                go.Scatter(
                    x=df["DATE_PERSIAN"],
                    y=df["WATER_TABLE_INTERPOLATE"],
                    mode='lines+markers',
                    name='Spline',
                    line=dict(
                        color='green',
                        width=2
                    ),
                    marker=dict(
                        color='green',
                        size=8,
                        opacity=1,
                    )
                ),
            )           

            fig.add_trace(
                go.Scatter(
                    x=df["DATE_PERSIAN"],
                    y=df["WATER_TABLE"],
                    mode='markers',
                    name='Original',
                    marker=dict(
                        color='black',
                        size=12,
                    )
                )
            )

            fig.update_layout(
                title=df["LOCATION_NAME"].unique()[0],
                yaxis_title="Water Table (m)",
                font=dict(
                    family="Courier New, monospace",
                    size=18,
                    color="RebeccaPurple"
                )
            )

            fig.show()
